# Tutorial Dataset

Before working on a dataset it's always a good idea to get an overview of it's structure, content and properties, no
matter if you acquired it yourself or use publicly available open data (e.g., from [www.openneuro.org](https://openneuro.org) or in the case of this tutorial from [Zeng et al.](https://link.springer.com/article/10.1007%2Fs12021-013-9187-0) which can be downloaded [here](http://fcon_1000.projects.nitrc.org/indi/retro/BeijingEOEC.html)). </br>

Always remember: a brief check at the beginning can save you tons of errors and hours of extra work!

#### Dataset structure & content
Starting with a brief look at our dataset, we can use the [linux inbuilt command `tree`](https://linux.die.net/man/1/tree) that will list the structure & content of our dataset in a tree-like format.

In [ ]:
!tree /data/dataset

As you can see we have three participants, each containing one anatomical image and two functional series. 
Now, if we want to know more about those files in terms of properties, we can use the `nib-ls` command which is part of the very useful [nibabel python package](http://nipy.org/nibabel/) (more on that in the next session).

In [ ]:
!nib-ls /data/dataset/*/*/*/*

#### The Brain Imaging Data Structure (BIDS) format
As you might have noticed: our dataset is highly structured. While this seems normal, it unfortunately isn't. Everyone uses different ways of storing and managing data, even within labs. [BIDS](http://bids.neuroimaging.io) is intendend to change that via propposing a certain structure and type of data that is easy to grasp & machine readible.

The main idea is to follow a strict set of rules (graphic taken from [here](https://www.nature.com/articles/sdata201644/figures/1)):

<img src="../slides/images/bids.jpg" alt="Drawing" style="width: 800px;"/>

Spending a closer look, you can see that it all starts with "my_dataset" folder which includes the dataset in BIDS: </br>

- Within that we have a file called "participants.tsv" contain information about the dataset's participants (e.g., demographics like sex, age, handedness, etc.). 
- Furthermore, we have the particpant corresponding folders "sub-**", which shows you the so called participant identifier (e.g., 01, 02, etc.). 
- Within each participant folder you'll find the respective acquired/included neuroimaging data, which is structured by type of data: </br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; "anat" -> anatomical images, "func" -> functional images, "dwi" -> diffusion weighted images.  
- Within those you can see divers files: neuroimaging data (ending with .nii.gz), JSON sidecar files (ending with .json), etc. . 
    - the neuroimaging files include a modality identifier: &nbsp; `*_T1w --> T1 weighted`, `*_bold --> bold weighted`
    - in case of task neuroimaging data, files include a task identifier: &nbsp; `*task-rest*` 

##### How to make use of it

BIDS basically describes how you should organize and structure your data, which not only helps you, but also others when sharing your data (which is also eased up). This also allows hassle free applications of other workflows and pipelines which can work with BIDS datasets, increases reproducibility and simplifies collaboration. 
Besides these advantages there are a lot of tools which are intended to and ease up the work with BIDS datasets. A very good example is [pybids](https://github.com/INCF/pybids), which is incredibly useful for any kind of interaction with [BIDS](http://bids.neuroimaging.io) datasets, e.g. within a nice & reproducible [nipype](https://github.com/nipy/nipype) [workflow](http://nbviewer.jupyter.org/github/nipy/workshops/blob/master/170327-nipype/notebooks/basic-bids/basic_data_input_bids.ipynb). Make sure to also have a look at [bidsutils](https://github.com/INCF/bidsutils). Furthermore, you should also check the already mentioned [BIDS apps](http://bids-apps.neuroimaging.io). These are "portable neuroimaging pipelines that understand BIDS datasets". More precisely, [BIDS apps](http://bids-apps.neuroimaging.io/about/) are neuroimaging pipelines / workflows for a [huge variety of analyses](http://bids-apps.neuroimaging.io/apps/) packed in a docker image that will work / run out of the box given a [BIDS dataset](https://www.nature.com/articles/sdata201644) as input. It won't get any more comfortable (okay, maybe with [openneuro.org](https://openneuro.org)).

That being said, let's actually check out our dataset in even more detail. 

**Note**: if you decide to use any of the mentioned tools please show credit by citing them: [BIDS](https://doi.org/doi:10.1038/sdata.2016.44), [BIDS apps](https://doi.org/10.1371/journal.pcbi.1005209).


#### PyBIDS

PyBIDS is a Python library to centralize interactions with datasets conforming BIDS format. It builds upon [Grabbit](https://github.com/grabbles/grabbit), which is a lightweight Python 2 and 3 package for simple queries over filenames within a project. It's geared towards projects or applications with highly structured filenames that allow useful queries to be performed without having to inspect the file metadata or contents. The included [grabbids](https://github.com/INCF/pybids/tree/master/bids/grabbids) is a BIDS-specific extension of grabbit that makes it particularly easy to work with BIDS projects, and provides additional functionality.

We can initialize a grabbids Layout object like so:

In [ ]:
from bids.grabbids import BIDSLayout
project_root = 'data/dataset'
layout = BIDSLayout(project_root)

Once we've initialized a Layout, we can do simple things like getting a list of subject labels:

In [ ]:
layout.get_subjects()

Counting is kind of trivial; everyone can count! More usefully, we can run simple logical queries, returning the results in a variety of formats:

In [ ]:
files = layout.get(subject='0[12]', extensions='.nii.gz')

In [ ]:
files[0]

In the above snippet, we retrieve all files with subject id 1 or 2 (notice that any entity defined in the config file can be used a filtering argument), and with a file extension of .nii.gz. The returned result is a list of named tuples, one per file, allowing direct access to the defined entities as attributes.

Some other examples of get() requests:

In [ ]:
layout.get(target='type', return_type='id', subject=1)

Get all metadata for a given file

In [ ]:
layout.get_metadata('data/dataset/sub-01/func/sub-01_task-rest-EC_bold.niig.gz')["RepetitionTime"]